# Generación de Datos Maestros de Clientes - ETL

**Autor:** [Tu Nombre]  
**Fecha:** 2026-02-21  
**Versión:** 1.0

## 📋 Descripción General
Este notebook ejecuta el proceso completo de Extracción, Transformación y Carga (ETL) de datos maestros de clientes bancarios. Genera 100 registros sintéticos de clientes con datos realistas usando Faker, los transforma a través de tres zonas de datos (Raw, Curada y Productiva) aplicando estandarizaciones bancarias.

## 🎯 Objetivos
1. Conectar a la base de datos PostgreSQL local con credenciales configuradas.
2. Generar 100 registros de clientes sintéticos con datos bancarios realistas.
3. Aplicar transformaciones y mapeos de columnas según estándares bancarios.
4. Cargar datos en tres zonas: Raw (ZR), Curada (ZC) y Productiva (ZP).
5. Crear dimensión de clientes optimizada para análisis.
6. Liberar recursos del sistema al finalizar.

## ⚙️ Requisitos Previos
* **Librerías:** `pandas`, `numpy`, `sqlalchemy`, `python-dotenv`, `faker`.
* **Base de Datos:** PostgreSQL configurada con conexión local.
* **Variables de entorno:** `.env` con credenciales (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME).
* **Esquemas en PostgreSQL:** `zr_cli`, `zc_cli`, `zp` (deben existir).
* **Archivos generados:** CSV en `data/raw/`, `data/curada/`, `data/productiva/`.

## 📊 Flujo de Datos
```
Generación Faker (100 clientes sintéticos)
    ↓
[Raw Zone] → CSV y tabla td_datos_clientes → zr_cli.zr_fake_cli_datos_clientes
    ↓
[Curada Zone] → Transformación y normalización → zc_cli.zc_cli_datos_clientes
    ↓
[Productiva Zone] → Dimensión optimizada → zp.td_datos_clientes
    ↓
Exportación a CSV en zona productiva
```

## 🔑 Campos Principales Generados
- `codigoSecuencialCliente`: Identificador secuencial (1000-1099)
- `codigoIdentificacionCliente`: Código único (CUS-XXXXX)
- `tipoIdentificacionCliente`: CÉDULA, RUC o PASAPORTE
- `numeroIdentificacionCliente`: Número único de ID
- `nombreCompletoCliente`: Nombre del cliente (generado con Faker)
- `segmentoCliente`: RETAIL, CORPORATIVO, PYME, WEALTH
- `scoreCrediticioCliente`: Score entre 300 y 1000
- `provinciaCliente`: Pichincha, Guayas, Azuay, Manabí, Loja
- `ciudadCliente`: Quito, Guayaquil, Cuenca, Manta
- `fechaRegistroCliente`: Fecha aleatoria (últimos 5 años)

## ⚠️ Notas Importantes
- Los datos son 100% sintéticos para POC/testing
- Utiliza Faker con seed=42 para reproducibilidad
- Implementa transformaciones case-insensitive (mayúsculas)
- Las fechas se convierten a formato DATE en PostgreSQL para consistencia

In [189]:
from IPython.display import HTML, display

def aplicar_estilo_bancario():
    style = """
    <style>
        .dataframe {
            border-collapse: collapse;
            border: 2px solid #1a237e; /* Azul Marino Bancario */
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            font-size: 12px;
        }
        .dataframe thead {
            background-color: #1a237e;
            color: white;
            text-align: center;
        }
        .dataframe th, .dataframe td {
            padding: 10px 15px;
            border: 1px solid #e0e0e0;
        }
        .dataframe tbody tr:nth-child(even) {
            background-color: #f5f5f5;
        }
        .dataframe tbody tr:hover {
            background-color: #e8eaf6; /* Resaltado suave al pasar el mouse */
            cursor: pointer;
        }
        .dataframe td {
            text-align: right;
        }
        /* Alinear a la izquierda columnas de texto */
        .dataframe td:nth-child(2), .dataframe td:nth-child(3) {
            text-align: left;
        }
    </style>
    """
    display(HTML(style))

aplicar_estilo_bancario()

## Paso 1: Configuración de Estilos Visuales

Aplica estilos CSS personalizados a los DataFrames para que se visualicen con colores bancarios. Esto mejora la legibilidad de los datos durante el análisis exploratorio.

**Características:**
- Encabezados con fondo azul marino (#1a237e)
- Filas alternas con fondo gris para mejor contraste
- Efecto hover para interactividad
- Fuente Segoe UI con tamaño optimizado


## Paso 2: Importación de Librerías Requeridas

Carga todas las dependencias necesarias para el procesamiento ETL:

| Librería | Propósito |
|----------|----------|
| `pandas` | Manipulación y análisis de datos (DataFrames) |
| `numpy` | Operaciones numéricas y generación de datos aleatorios |
| `datetime` | Manejo de fechas y cálculos temporales |
| `faker` | Generación de datos sintéticos realistas |
| `sqlalchemy` | ORM y conexión con PostgreSQL |
| `dotenv` | Carga segura de credenciales desde archivo `.env` |
| `urllib.parse` | Codificación de contraseñas con caracteres especiales |

In [190]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
from sqlalchemy import create_engine, text, MetaData, Table, select
import os
from dotenv import load_dotenv
import urllib.parse
import logging
import gc
from pathlib import Path

# Configurar logging estructurado
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('../logs/etl_clientes.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

## Paso 3: Configuración de Parámetros Globales

Define las variables que controlan el comportamiento del notebook:

**Parámetros de Datos:**
- `N_CLIENTES = 100`: Cantidad de clientes sintéticos a generar
- `fake = Faker(['es_MX'])`: Generador con localización en español mexicano

**Esquemas y Tablas en PostgreSQL:**
- **Zona Raw (ZR):** `zr_cli.zr_fake_cli_datos_clientes` (datos sin transformar)
- **Zona Curada (ZC):** `zc_cli.zc_cli_datos_clientes` (datos normalizados)
- **Zona Productiva (ZP):** `zp.td_datos_clientes` (dimensión final de clientes)

In [191]:
# Parámetros del POC
N_CLIENTES = 500
# Inicializamos Faker con localización en español
fake = Faker(['es_MX'])
# Nombres para objetos 
#################################
## ZONA RAW
#################################

nombre_esquema_zr_cli = 'zr_cli'
nombre_tabla_zr_cli = 'zr_fake_cli_datos_clientes'

#################################
## ZONA CURADA
#################################

nombre_esquema_zc_cli = 'zc_cli'
nombre_tabla_zc_cli = 'zc_cli_datos_clientes'

#################################
## ZONA PRODUCTIVA
#################################

nombre_esquema_zp = 'zp'
nombre_tabla_zp_cli = 'td_datos_clientes'


## Paso 4: Conexión a PostgreSQL

Establece la conexión segura con la base de datos PostgreSQL usando credenciales del archivo `.env`.

**Proceso:**
1. Carga variables de entorno (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)
2. Codifica la contraseña con `urllib.parse.quote_plus()` para manejar caracteres especiales
3. Crea engine de SQLAlchemy con la URL codificada
4. Valida la conexión con un intento de conexión

**Ejemplo de `.env`:**
```
DB_USER=usuario
DB_PASSWORD=contraseña@especial.123
DB_HOST=localhost
DB_PORT=5432
DB_NAME=banco_poc
```

In [192]:
# Forzar recarga del .env
load_dotenv(override=True)

try:
    # Validar que existan las credenciales necesarias
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")
    db = os.getenv("DB_NAME")
    
    # Validar credenciales no nulas
    if not all([user, password, host, port, db]):
        missing = [k for k, v in {
            'DB_USER': user,
            'DB_PASSWORD': password,
            'DB_HOST': host,
            'DB_PORT': port,
            'DB_NAME': db
        }.items() if not v]
        raise ValueError(f"Variables de entorno faltantes: {', '.join(missing)}")
    
    logger.info("✅ Variables de entorno cargadas correctamente")
    
    # Codificar la contraseña para caracteres especiales
    password_encoded = urllib.parse.quote_plus(password)
    
    # Crear engine con parámetros de seguridad
    engine = create_engine(
        f'postgresql://{user}:{password_encoded}@{host}:{port}/{db}',
        connect_args={
            'connect_timeout': 10,
            'sslmode': 'prefer'
        },
        echo=False
    )
    
    # Probar conexión
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    logger.info("✅ Conexión exitosa con PostgreSQL")
    
except ValueError as e:
    logger.error(f"❌ Error de configuración: {e}")
    raise
except Exception as e:
    logger.error(f"❌ Error al conectar a BD: {e}")
    raise

2026-02-21 19:17:59,688 - __main__ - INFO - ✅ Variables de entorno cargadas correctamente
2026-02-21 19:17:59,743 - __main__ - INFO - ✅ Conexión exitosa con PostgreSQL


## Paso 5: Generación de Datos Sintéticos

### Paso 5a: Generación de Dataset de Clientes Sintético

Crea 100 registros de clientes con datos realistas usando Faker.

**Lógica de Generación:**

**1. Función `generar_id_negocio(tipo)`:**
- Cédula: 10 dígitos aleatorios
- RUC: 10 dígitos + sufijo '001' (estándar ecuatoriano)
- Pasaporte: 'P' + 8 dígitos aleatorios

**2. Dimensión de Clientes (100 registros):**
- `cliente_id`: Secuencial 1000-1099
- `codigo_unico_cliente`: Formato CUS-XXXXX
- `nombre`: Generado con Faker en español mexicano
- `email`: Correos generados (ascii free email)
- `telefono`: Números telefónicos generados
- `tipo_identificacion`: 80% Cédula, 15% RUC, 5% Pasaporte
- `identificacion`: ID único según tipo
- `segmento`: RETAIL (40%), CORPORATIVO, PYME, WEALTH
- `provincia`: Pichincha, Guayas, Azuay, Manabí, Loja
- `ciudad`: Quito, Guayaquil, Cuenca, Manta
- `fecha_registro`: Aleatorio en últimos 5 años
- `score_crediticio`: Entero entre 300 y 1000

**3. Campos de Auditoría:**
- `periodo`: Formato YYYYMM (fecha actual)
- `fecha_proceso`: Fecha actual (solo fecha)
- `fecha_ingesta`: Timestamp completo con microsegundos

In [193]:
def generar_dataset_clientes():
    """
    Genera dataset de clientes sintéticos con datos realistas.
    
    Returns:
        pd.DataFrame: DataFrame con 100 clientes y campos de auditoría
        
    Validaciones:
        - IDs únicos verificados
        - Emails con formato válido
        - Scores en rango válido (300-1000)
        - Sin valores NULL
    """
    np.random.seed(42)
    Faker.seed(42)
    logger.info(f"Iniciando generación de {N_CLIENTES} clientes sintéticos")
    
    try:
        def generar_id_negocio(tipo: str) -> str:
            """
            Genera ID de negocio según tipo.
            
            Args:
                tipo: CÉDULA, RUC o PASAPORTE
                
            Returns:
                str: ID único formateado
            """
            if tipo == 'CÉDULA':
                return ''.join(np.random.choice(list('0123456789'), 10))
            elif tipo == 'RUC':
                return ''.join(np.random.choice(list('0123456789'), 10)) + '001'
            else:
                return 'P' + ''.join(np.random.choice(list('0123456789'), 8))

        # Generar tipos de ID con distribución realista
        tipos_id = np.random.choice(
            ['CÉDULA', 'RUC', 'PASAPORTE'], 
            N_CLIENTES, 
            p=[0.8, 0.15, 0.05]
        )
        
        # Crear DataFrame con datos
        clientes = pd.DataFrame({
            'cliente_id': range(1000, 1000 + N_CLIENTES),
            'codigo_unico_cliente': [f"CUS-{i:05d}" for i in range(1, N_CLIENTES + 1)],
            'nombre': [fake.name().upper() for _ in range(N_CLIENTES)],
            'email': [fake.ascii_free_email() for _ in range(N_CLIENTES)],
            'telefono': [fake.phone_number() for _ in range(N_CLIENTES)],
            'tipo_identificacion': tipos_id,
            'identificacion': [generar_id_negocio(t) for t in tipos_id],
            'segmento': np.random.choice(['RETAIL', 'CORPORATIVO', 'PYME', 'WEALTH'], N_CLIENTES),
            'provincia': np.random.choice(['Pichincha', 'Guayas', 'Azuay', 'Manabí', 'Loja'], N_CLIENTES),
            'ciudad': np.random.choice(['Quito', 'Guayaquil', 'Cuenca', 'Manta'], N_CLIENTES),
            'fecha_registro': [fake.date_between(start_date='-5y', end_date='today') for _ in range(N_CLIENTES)],
            'score_crediticio': np.random.randint(300, 1000, N_CLIENTES)
        })
        
        # Validaciones de calidad de datos
        assert len(clientes) == N_CLIENTES, f"Cantidad de registros incorrecta: {len(clientes)}"
        assert clientes['cliente_id'].is_unique, "IDs de cliente duplicados"
        assert clientes['codigo_unico_cliente'].is_unique, "Códigos únicos duplicados"
        assert clientes.isnull().sum().sum() == 0, "Hay valores NULL en los datos"
        assert (clientes['score_crediticio'] >= 300).all(), "Scores por debajo de 300"
        assert (clientes['score_crediticio'] < 1000).all(), "Scores fuera de rango"
        
        logger.info(f"✅ {N_CLIENTES} clientes generados exitosamente")
        logger.info(f"  - IDs únicos verificados: {clientes['cliente_id'].is_unique}")
        logger.info(f"  - Sin valores NULL: {clientes.isnull().sum().sum() == 0}")
        
        return clientes
        
    except AssertionError as e:
        logger.error(f"❌ Validación de datos fallida: {e}")
        raise
    except Exception as e:
        logger.error(f"❌ Error al generar dataset de clientes: {e}")
        raise

df_clientes = generar_dataset_clientes()

# Agregar campos de auditoría
try:
    fechaActual = pd.Timestamp.now()
    df_clientes['periodo'] = fechaActual.strftime('%Y%m')
    df_clientes['fecha_proceso'] = fechaActual.date()
    df_clientes['fecha_ingesta'] = fechaActual
    
    logger.info(f"Campos de auditoría agregados para período {fechaActual.strftime('%Y%m%d')}")
    logger.info(f"Shape del DataFrame: {df_clientes.shape}")
    
except Exception as e:
    logger.error(f"❌ Error al agregar campos de auditoría: {e}")
    raise

2026-02-21 19:17:59,782 - __main__ - INFO - Iniciando generación de 500 clientes sintéticos
2026-02-21 19:17:59,868 - __main__ - INFO - ✅ 500 clientes generados exitosamente
2026-02-21 19:17:59,870 - __main__ - INFO -   - IDs únicos verificados: True
2026-02-21 19:17:59,873 - __main__ - INFO -   - Sin valores NULL: True
2026-02-21 19:17:59,889 - __main__ - INFO - Campos de auditoría agregados para período 20260221
2026-02-21 19:17:59,893 - __main__ - INFO - Shape del DataFrame: (500, 15)


## Paso 6: Exportación a CSV - Zona Raw

Guarda el dataset sin transformar en archivo CSV para auditoría y respaldo.

**Destino:** `data/raw/zr_fake_cli_datos_clientes.csv`

**Propósito de la Zona Raw:**
- Mantener copia exacta de datos originales generados
- Facilitar auditoría de cambios posteriores
- Permitir reintentos de procesamiento si falla la transformación

In [194]:
try:
    # Crear directorio si no existe
    raw_path = Path('../data/raw')
    raw_path.mkdir(parents=True, exist_ok=True)
    
    # Guardar archivo con validación
    csv_path = raw_path / f'{nombre_tabla_zr_cli}.csv'
    df_clientes.to_csv(csv_path, index=False)
    
    # Validar archivo creado
    if not csv_path.exists():
        raise FileNotFoundError(f"Archivo no se creó: {csv_path}")
    
    file_size = csv_path.stat().st_size / 1024  # KB
    logger.info(f"✅ Archivo guardado: {csv_path} ({file_size:.2f} KB)")
    
except Exception as e:
    logger.error(f"❌ Error al exportar CSV (Raw): {e}")
    raise

2026-02-21 19:17:59,924 - __main__ - INFO - ✅ Archivo guardado: ../data/raw/zr_fake_cli_datos_clientes.csv (86.94 KB)


## Paso 7: Función Auxiliar para Carga en PostgreSQL

Define `cargar_datos_postgresql()` para persistir datos en diferentes esquemas.

**Parámetros:**
- `df`: DataFrame a cargar
- `nombre_esquema`: Schema destino (zr_cli, zc_cli, zp)
- `nombre_tabla`: Tabla destino
- `primary_key_column`: Columna clave primaria

**Lógica:**
1. Carga DataFrame con `to_sql()` en modo 'replace' (DROP + CREATE)
2. Define clave primaria en la tabla creada
3. Manejo robusto de excepciones

**Nota:** Usa 'replace' para POC; en producción usar 'append' + particionamiento

In [195]:
def cargar_datos_postgresql(df: pd.DataFrame, nombre_esquema: str, nombre_tabla: str, primary_key_column: str) -> bool:
    """
    Carga datos en PostgreSQL con validación y manejo de errores.
    
    Args:
        df: DataFrame a cargar
        nombre_esquema: Schema destino
        nombre_tabla: Tabla destino
        primary_key_column: Columna para clave primaria
        
    Returns:
        bool: True si la carga fue exitosa
        
    Raises:
        ValueError: Si los parámetros son inválidos
        Exception: Si hay error en la carga
    """
    try:
        # Validar inputs
        if df.empty:
            raise ValueError("DataFrame vacío")
        if not isinstance(nombre_esquema, str) or not isinstance(nombre_tabla, str):
            raise ValueError("Nombre de esquema o tabla inválido")
        
        # Validar que la columna PK existe
        if primary_key_column.strip('"') not in df.columns:
            raise ValueError(f"Columna PK no existe: {primary_key_column}")
        
        logger.info(f"Iniciando carga: {nombre_esquema}.{nombre_tabla}")
        logger.info(f"  - Registros: {len(df)}")
        logger.info(f"  - Columnas: {df.shape[1]}")
        
        # Carga a PostgreSQL
        df.to_sql(nombre_tabla, engine, schema=nombre_esquema, if_exists='replace', index=False)
        logger.info(f"✅ Tabla creada: {nombre_esquema}.{nombre_tabla}")
        
        # Agregar clave primaria
        with engine.connect() as con:
            query = text(f'ALTER TABLE {nombre_esquema}.{nombre_tabla} ADD PRIMARY KEY ({primary_key_column});')
            con.execute(query)
            con.commit()
        
        logger.info(f"✅ Clave primaria agregada: {primary_key_column}")
        logger.info(f"✅ Carga completada exitosamente")
        return True
        
    except ValueError as e:
        logger.error(f"❌ Error de validación: {e}")
        raise
    except Exception as e:
        logger.error(f"❌ Error al cargar en PostgreSQL: {e}")
        raise

## Paso 8: Carga a PostgreSQL - Zona Raw

Inserta los datos sin transformar en la tabla `zr_cli.zr_fake_cli_datos_clientes`.

**Resultado:**
- Tabla con 100 registros de clientes
- Clave primaria: cliente_id
- Esquema: zr_cli (Zona Raw Clientes)

In [196]:
try:
    logger.info(f"Cargando datos a Zona Raw...")
    cargar_datos_postgresql(
        df_clientes,
        nombre_esquema_zr_cli,
        nombre_tabla_zr_cli,
        'cliente_id'
    )
except Exception as e:
    logger.error(f"❌ Fallo en carga a Zona Raw: {e}")
    raise

2026-02-21 19:18:00,007 - __main__ - INFO - Cargando datos a Zona Raw...
2026-02-21 19:18:00,008 - __main__ - INFO - Iniciando carga: zr_cli.zr_fake_cli_datos_clientes
2026-02-21 19:18:00,018 - __main__ - INFO -   - Registros: 500
2026-02-21 19:18:00,019 - __main__ - INFO -   - Columnas: 15
2026-02-21 19:18:00,209 - __main__ - INFO - ✅ Tabla creada: zr_cli.zr_fake_cli_datos_clientes
2026-02-21 19:18:00,221 - __main__ - INFO - ✅ Clave primaria agregada: cliente_id
2026-02-21 19:18:00,225 - __main__ - INFO - ✅ Carga completada exitosamente


## Paso 9: Transformación a Zona Curada (ZC)

Normaliza los datos aplicando reglas de negocio y estandarización de nomenclatura bancaria.

**Proceso:**

**1. Extracción desde Raw:**
- Refleja tabla desde `zr_cli.zr_fake_cli_datos_clientes` usando SQLAlchemy MetaData
- Lee estructura y datos del archivo original

**2. Mapeo de Columnas (Nomenclatura bancaria):**
- `periodo` → `codigoPeriodo`
- `cliente_id` → `codigoSecuencialCliente`
- `codigo_unico_cliente` → `codigoIdentificacionCliente`
- `tipo_identificacion` → `tipoIdentificacionCliente`
- `identificacion` → `numeroIdentificacionCliente`
- `nombre` → `nombreCompletoCliente`
- `email` → `correoElectronicoCliente`
- `telefono` → `telefonoCliente`
- `segmento` → `segmentoCliente`
- `score_crediticio` → `scoreCrediticioCliente`
- `provincia` → `provinciaCliente`
- `ciudad` → `ciudadCliente`
- `fecha_registro` → `fechaRegistroCliente`
- `fecha_proceso` → `periodo`

**3. Transformaciones de Datos:**
- Convierte a MAYÚSCULAS: provincia, ciudad, tipo_identificación
- Limpia espacios en blanco: `.str.strip()`
- Reordena columnas en orden lógico

**4. Campos de Auditoría:**
- Agrega `fechaIngesta`: Timestamp completo

**Resultado:** DataFrame normalizado con nomenclatura bancaria estándar.

In [197]:
try:
    logger.info("Iniciando transformación a Zona Curada")
    
    # Crear un objeto MetaData
    metadata = MetaData()
    
    # Reflejar la tabla desde PostgreSQL
    tabla_clientes = Table(
        nombre_tabla_zr_cli, 
        metadata, 
        autoload_with=engine, 
        schema=nombre_esquema_zr_cli
    )
    logger.info(f"✅ Tabla reflejada: {nombre_esquema_zr_cli}.{nombre_tabla_zr_cli}")
    
    # Construir consulta
    stmt = select(tabla_clientes)
    
    # Cargar en Pandas
    with engine.connect() as con:
        df_db_clientes = pd.read_sql(stmt, con)
    logger.info(f"✅ {len(df_db_clientes)} registros cargados desde PostgreSQL")
    
    # Mapeo de columnas
    mapping = {
        'periodo': 'codigoPeriodo', 
        'cliente_id': 'codigoSecuencialCliente',
        'codigo_unico_cliente': 'codigoIdentificacionCliente',
        'tipo_identificacion': 'tipoIdentificacionCliente',
        'identificacion': 'numeroIdentificacionCliente',
        'nombre': 'nombreCompletoCliente',
        'email': 'correoElectronicoCliente',
        'telefono': 'telefonoCliente',
        'segmento': 'segmentoCliente',
        'score_crediticio': 'scoreCrediticioCliente',
        'provincia': 'provinciaCliente',
        'ciudad': 'ciudadCliente',
        'fecha_registro': 'fechaRegistroCliente',
        'fecha_proceso': 'periodo'
    }
    
    df_db_clientes = df_db_clientes.rename(columns=mapping)
    logger.info(f"✅ Columnas renombradas: {len(mapping)} campos mapeados")
    
    # Transformaciones con validación
    columnas_upper = ['provinciaCliente', 'ciudadCliente', 'tipoIdentificacionCliente']
    for col in columnas_upper:
        if col in df_db_clientes.columns:
            # Manejar valores NULL
            df_db_clientes[col] = df_db_clientes[col].fillna('').str.upper().str.strip()
    
    # Limpiar espacios en blanco
    columnas_string = ['codigoIdentificacionCliente', 'nombreCompletoCliente', 
                       'correoElectronicoCliente', 'segmentoCliente']
    for col in columnas_string:
        if col in df_db_clientes.columns:
            df_db_clientes[col] = df_db_clientes[col].fillna('').str.strip()
    
    logger.info("✅ Transformaciones aplicadas")
    
    # Reordenar columnas
    columnas_finales = [
        'codigoSecuencialCliente',
        'codigoPeriodo',
        'codigoIdentificacionCliente',
        'tipoIdentificacionCliente',
        'numeroIdentificacionCliente',
        'nombreCompletoCliente',
        'correoElectronicoCliente',
        'telefonoCliente',
        'segmentoCliente',
        'scoreCrediticioCliente',
        'provinciaCliente',
        'ciudadCliente',
        'fechaRegistroCliente',
        'periodo'
    ]
    
    df_db_clientes = df_db_clientes[columnas_finales]
    df_db_clientes['fechaIngesta'] = fechaActual
    
    logger.info(f"✅ Transformación completada. Shape: {df_db_clientes.shape}")
    
except Exception as e:
    logger.error(f"❌ Error en transformación a ZC: {e}")
    raise

2026-02-21 19:18:00,258 - __main__ - INFO - Iniciando transformación a Zona Curada
2026-02-21 19:18:00,283 - __main__ - INFO - ✅ Tabla reflejada: zr_cli.zr_fake_cli_datos_clientes
2026-02-21 19:18:00,311 - __main__ - INFO - ✅ 500 registros cargados desde PostgreSQL
2026-02-21 19:18:00,319 - __main__ - INFO - ✅ Columnas renombradas: 14 campos mapeados
2026-02-21 19:18:00,324 - __main__ - INFO - ✅ Transformaciones aplicadas
2026-02-21 19:18:00,345 - __main__ - INFO - ✅ Transformación completada. Shape: (500, 15)


## Paso 10: Exportación a CSV - Zona Curada

Guarda los datos transformados en archivo CSV.

**Destino:** `data/curada/zc_cli_datos_clientes.csv`

**Propósito:**
- Respaldo de datos curados
- Verificación manual de transformaciones
- Punto de referencia para auditoría

In [198]:
try:
    # Crear directorio si no existe
    curada_path = Path('../data/curada')
    curada_path.mkdir(parents=True, exist_ok=True)
    
    csv_path = curada_path / f'{nombre_tabla_zc_cli}.csv'
    df_db_clientes.to_csv(csv_path, index=False)
    
    if not csv_path.exists():
        raise FileNotFoundError(f"Archivo no se creó: {csv_path}")
    
    file_size = csv_path.stat().st_size / 1024
    logger.info(f"✅ Archivo guardado: {csv_path} ({file_size:.2f} KB)")
    
except Exception as e:
    logger.error(f"❌ Error al exportar CSV (Curada): {e}")
    raise

2026-02-21 19:18:00,441 - __main__ - INFO - ✅ Archivo guardado: ../data/curada/zc_cli_datos_clientes.csv (87.05 KB)


## Paso 11: Carga a PostgreSQL - Zona Curada

Inserta los datos transformados en la tabla `zc_cli.zc_cli_datos_clientes`.

**Tabla destino:** `zc_cli.zc_cli_datos_clientes`
**Clave Primaria:** codigoSecuencialCliente

Esta es la zona de análisis intermedia con datos ya normalizados.

In [199]:
try:
    logger.info(f"Cargando datos a Zona Curada...")
    cargar_datos_postgresql(
        df_db_clientes,
        nombre_esquema_zc_cli,
        nombre_tabla_zc_cli,
        '"codigoSecuencialCliente"'
    )
except Exception as e:
    logger.error(f"❌ Fallo en carga a Zona Curada: {e}")
    raise

2026-02-21 19:18:00,500 - __main__ - INFO - Cargando datos a Zona Curada...
2026-02-21 19:18:00,506 - __main__ - INFO - Iniciando carga: zc_cli.zc_cli_datos_clientes
2026-02-21 19:18:00,507 - __main__ - INFO -   - Registros: 500
2026-02-21 19:18:00,508 - __main__ - INFO -   - Columnas: 15
2026-02-21 19:18:00,670 - __main__ - INFO - ✅ Tabla creada: zc_cli.zc_cli_datos_clientes
2026-02-21 19:18:00,681 - __main__ - INFO - ✅ Clave primaria agregada: "codigoSecuencialCliente"
2026-02-21 19:18:00,684 - __main__ - INFO - ✅ Carga completada exitosamente


## Paso 12: Generación de Dimensión de Clientes Optimizada

Selecciona y reordena columnas para crear dimensión final optimizada para análisis.

**Columnas Seleccionadas (12 campos):**
1. `codigoSecuencialCliente` - ID secuencial
2. `codigoPeriodo` - Período YYYYMM
3. `codigoIdentificacionCliente` - Código único cliente
4. `tipoIdentificacionCliente` - Tipo de ID
5. `numeroIdentificacionCliente` - Número de ID
6. `nombreCompletoCliente` - Nombre completo
7. `segmentoCliente` - Segmento cliente
8. `scoreCrediticioCliente` - Score crediticio
9. `provinciaCliente` - Provincia
10. `ciudadCliente` - Ciudad
11. `fechaRegistroCliente` - Fecha de registro
12. `periodo` - Período YYYYMMDD

**Propósito:** 
- Reducir columnas innecesarias (email, teléfono, etc.)
- Optimizar performance de consultas analíticas
- Crear dimensión lista para joins con hechos

In [200]:
try:
    logger.info("Generando dimensión optimizada de clientes")
    
    columnas_dimension = [
        'codigoSecuencialCliente',
        'codigoPeriodo',
        'codigoIdentificacionCliente',
        'tipoIdentificacionCliente',
        'numeroIdentificacionCliente',
        'nombreCompletoCliente',
        'segmentoCliente',
        'scoreCrediticioCliente',
        'provinciaCliente',
        'ciudadCliente',
        'fechaRegistroCliente',
        'periodo'
    ]
    
    # Validar que todas las columnas existen
    cols_faltantes = [c for c in columnas_dimension if c not in df_db_clientes.columns]
    if cols_faltantes:
        raise ValueError(f"Columnas faltantes: {cols_faltantes}")
    
    df_db_clientes = df_db_clientes[columnas_dimension]
    logger.info(f"✅ Dimensión generada: {df_db_clientes.shape}")
    
except Exception as e:
    logger.error(f"❌ Error al generar dimensión: {e}")
    raise

2026-02-21 19:18:00,707 - __main__ - INFO - Generando dimensión optimizada de clientes
2026-02-21 19:18:00,715 - __main__ - INFO - ✅ Dimensión generada: (500, 12)


## Paso 13: Exportación a CSV - Zona Productiva

Guarda la dimensión de clientes optimizada en archivo CSV.

**Destino:** `data/productiva/td_datos_clientes.csv`

**Contenido:** 
Dimensión de clientes con 12 columnas esenciales.

**Propósito:**
- Dataset final para análisis y reportes
- Base para consolidación con tablas de hechos (inversiones)
- Auditoría de dimensión entregada

In [201]:
try:
    # Crear directorio si no existe
    productiva_path = Path('../data/productiva')
    productiva_path.mkdir(parents=True, exist_ok=True)
    
    csv_path = productiva_path / f'{nombre_tabla_zp_cli}.csv'
    df_db_clientes.to_csv(csv_path, index=False)
    
    if not csv_path.exists():
        raise FileNotFoundError(f"Archivo no se creó: {csv_path}")
    
    file_size = csv_path.stat().st_size / 1024
    logger.info(f"✅ Archivo guardado: {csv_path} ({file_size:.2f} KB)")
    
except Exception as e:
    logger.error(f"❌ Error al exportar CSV (Productiva): {e}")
    raise

2026-02-21 19:18:00,772 - __main__ - INFO - ✅ Archivo guardado: ../data/productiva/td_datos_clientes.csv (54.94 KB)


## Paso 14: Carga a PostgreSQL - Zona Productiva

Inserta la dimensión final en la tabla `zp.td_datos_clientes` para consultas analíticas.

**Tabla destino:** `zp.td_datos_clientes` (Dimensión de Clientes)
**Registros:** 100 clientes
**Clave Primaria:** codigoSecuencialCliente

Esta es la tabla consultable para:
- Enriquecimiento de datos de inversiones
- Reportes de clientes por segmento
- Análisis de score crediticio
- Filtros geográficos (provincia, ciudad)

In [202]:
try:
    logger.info(f"Cargando datos a Zona Productiva...")
    cargar_datos_postgresql(
        df_db_clientes,
        nombre_esquema_zp,
        nombre_tabla_zp_cli,
        '"codigoSecuencialCliente"'
    )
    logger.info("✅ Dimensión de Clientes cargada exitosamente en PostgreSQL")
except Exception as e:
    logger.error(f"❌ Fallo en carga a Zona Productiva: {e}")
    raise

2026-02-21 19:18:00,812 - __main__ - INFO - Cargando datos a Zona Productiva...
2026-02-21 19:18:00,816 - __main__ - INFO - Iniciando carga: zp.td_datos_clientes
2026-02-21 19:18:00,826 - __main__ - INFO -   - Registros: 500
2026-02-21 19:18:00,833 - __main__ - INFO -   - Columnas: 12
2026-02-21 19:18:01,078 - __main__ - INFO - ✅ Tabla creada: zp.td_datos_clientes
2026-02-21 19:18:01,124 - __main__ - INFO - ✅ Clave primaria agregada: "codigoSecuencialCliente"
2026-02-21 19:18:01,127 - __main__ - INFO - ✅ Carga completada exitosamente
2026-02-21 19:18:01,128 - __main__ - INFO - ✅ Dimensión de Clientes cargada exitosamente en PostgreSQL


## Paso 15: Limpieza y Liberación de Recursos

Cierra conexiones de base de datos y libera memoria del kernel de Jupyter.

**Acciones:**
1. **Cierra conexiones PostgreSQL:**
   - Busca objetos con método `.close()` (conexiones DBAPI)
   - Busca objetos con método `.dispose()` (SQLAlchemy engines)

2. **Libera DataFrames:**
   - Identifica todos los DataFrames en memoria
   - Los elimina para liberar RAM

3. **Recolección de basura:**
   - Ejecuta `gc.collect()` para optimizar memoria

**Propósito:** 
- Evitar memory leaks en Jupyter
- Permitir nuevas ejecuciones del notebook sin problemas
- Liberar conexiones a PostgreSQL para otros procesos

**Resultado:** "🧹 Memoria RAM optimizada"

In [203]:
logger.info("--- Iniciando limpieza de recursos ---")

try:
    # 1. Cerrar conexiones a Bases de Datos
    possible_conn_names = ['conn', 'connection', 'db_conn', 'engine']
    
    for name in possible_conn_names:
        if name in globals():
            obj = globals()[name]
            try:
                if hasattr(obj, 'close'):
                    obj.close()
                    logger.info(f"✅ Conexión '{name}' cerrada")
                elif hasattr(obj, 'dispose'):
                    obj.dispose()
                    logger.info(f"✅ Engine '{name}' dispuesto")
            except Exception as e:
                logger.warning(f"⚠️ No se pudo cerrar '{name}': {e}")
    
    logger.info("🔌 Conexión a PostgreSQL cerrada correctamente")
    
    # 2. Eliminar DataFrames para liberar memoria
    df_variables = [var for var, obj in globals().items() 
                    if isinstance(obj, pd.DataFrame) and not var.startswith('_')]
    
    if df_variables:
        logger.info(f"Eliminando {len(df_variables)} DataFrames: {', '.join(df_variables)}")
        for var in df_variables:
            del globals()[var]
        logger.info("🗑️ DataFrames eliminados de la memoria")
    else:
        logger.info("No se encontraron DataFrames para eliminar")
    
    # 3. Forzar recolección de basura
    gc.collect()
    logger.info("✅ Recolección de basura completada")
    logger.info("🧹 Memoria RAM optimizada")
    logger.info("--- Limpieza finalizada ---")
    
except Exception as e:
    logger.error(f"❌ Error durante limpieza: {e}")
    raise

2026-02-21 19:18:01,225 - __main__ - INFO - --- Iniciando limpieza de recursos ---
2026-02-21 19:18:01,243 - __main__ - INFO - ✅ Conexión 'conn' cerrada
2026-02-21 19:18:01,255 - __main__ - INFO - ✅ Engine 'engine' dispuesto
2026-02-21 19:18:01,263 - __main__ - INFO - 🔌 Conexión a PostgreSQL cerrada correctamente
2026-02-21 19:18:01,274 - __main__ - INFO - Eliminando 2 DataFrames: df_clientes, df_db_clientes


2026-02-21 19:18:01,279 - __main__ - INFO - 🗑️ DataFrames eliminados de la memoria
2026-02-21 19:18:01,660 - __main__ - INFO - ✅ Recolección de basura completada
2026-02-21 19:18:01,663 - __main__ - INFO - 🧹 Memoria RAM optimizada
2026-02-21 19:18:01,665 - __main__ - INFO - --- Limpieza finalizada ---
